## Dependencias 

In [1]:
import pandas as pd 
import re
from keplergl import KeplerGl

/Users/josegustavofuentescabrera/Documents/anahuac/shiny-adventure/.venv/lib/python3.11/site-packages/keplergl/keplergl.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


## Carga de datos 

In [2]:
df = pd.read_parquet('../data/austin.parquet').sample(frac=0.05).reset_index(drop=True)

display(df.head())


df.shape


,start_time,dim_A,dim_B,dim_susc_type,dim_bike_type,mt_duration,mt_trips
0,2017-03-17 19:27:04,"(30.2659, -97.76822)","(30.2659, -97.76822)",Walk Up,classic,44,1
1,2023-11-03 12:54:09,"(30.264327, -97.736446)","(30.264327, -97.736446)",Student Membership,electric,24,1
2,2021-05-31 14:40:36,"(30.26896, -97.72843)","(30.25457, -97.74258)",Local365,electric,17,1
3,2016-06-17 20:31:34,"(30.27041, -97.75046)","(30.26896, -97.72843)",Walk Up,classic,19,1
4,2022-09-20 22:19:18,"(30.283, -97.7375)","(30.28395, -97.74198)",Student Membership,electric,3,1


(102361, 7)

In [3]:


# Elimina paréntesis y espacios, luego convierte a tupla y separa en columnas
def str_to_tuple(s):
    # Elimina paréntesis y espacios
    s_clean = re.sub(r'[\(\)\s]', '', str(s))
    # Divide por coma y convierte a float si posible
    parts = s_clean.split(',')
    if len(parts) == 2:
        try:
            return (float(parts[0]), float(parts[1]))
        except ValueError:
            return (float('nan'), float('nan'))
    else:
        return (float('nan'), float('nan'))

df[['start_lat','start_lon']] = df['dim_A'].apply(str_to_tuple).apply(pd.Series)
df[['end_lat','end_lon']] = df['dim_B'].apply(str_to_tuple).apply(pd.Series)

display(df.head())

df.shape


,start_time,dim_A,dim_B,dim_susc_type,dim_bike_type,mt_duration,mt_trips,start_lat,start_lon,end_lat,end_lon
0,2017-03-17 19:27:04,"(30.2659, -97.76822)","(30.2659, -97.76822)",Walk Up,classic,44,1,30.265900,-97.768220,30.265900,-97.768220
1,2023-11-03 12:54:09,"(30.264327, -97.736446)","(30.264327, -97.736446)",Student Membership,electric,24,1,30.264327,-97.736446,30.264327,-97.736446
2,2021-05-31 14:40:36,"(30.26896, -97.72843)","(30.25457, -97.74258)",Local365,electric,17,1,30.268960,-97.728430,30.254570,-97.742580
3,2016-06-17 20:31:34,"(30.27041, -97.75046)","(30.26896, -97.72843)",Walk Up,classic,19,1,30.270410,-97.750460,30.268960,-97.728430
4,2022-09-20 22:19:18,"(30.283, -97.7375)","(30.28395, -97.74198)",Student Membership,electric,3,1,30.283000,-97.737500,30.283950,-97.741980


(102361, 11)

In [4]:

# Solo las columnas necesarias para ARC y filtro de datetime
df_map = df.rename(columns={'start_time': 'datetime'})[['datetime', 'start_lat', 'start_lon', 'end_lat', 'end_lon']].copy()

mapa = KeplerGl(
    width=1920,
    height=1080
)

mapa.add_data(
    data=df_map,
    name="Viaje"
)

# Ajusta el config SOLO si existen las claves necesarias
config = mapa.config

# Centra el mapa y ajusta el estilo solo si la estructura existe
if 'config' in config:
    if 'mapState' in config['config']:
        config['config']['mapState'].update({
            "latitude": 30.2672,
            "longitude": -97.7431,
            "zoom": 11,
            "pitch": 0,
            "bearing": 0
        })
    if 'mapStyle' in config['config']:
        config['config']['mapStyle'].update({
            "styleType": "dark"
        })
    mapa.config = config  # Asigna el config modificado de vuelta

mapa.save_to_html(file_name="mapa_kepler.html", read_only=False,)


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to mapa_kepler.html!
